Задача с качеством уроков

В нашей школе ученики и репетиторы занимаются в специальном онлайн-классе, в котором они могут общаться друг с другом, рисовать на доске, переписываться в чате, обмениваться файлами и решать различные задачки. Информацию о каждом уроке мы старательно записываем в базу данных в таблицу lessons. На каждом таком уроке присутствует один репетитор и один ученик (существуют еще тренировочные уроки, у которых единственный участник – репетитор, но для нашей задачи они неинтересны). В конце урока, когда учитель и ученик покидают класс, они могут оценить (а могут и пропустить этот этап) качество пройденного урока по пятибальной шкале. Все оценки мы записываем в таблицу quality, чтобы мы могли узнавать, хорошо ли работает наш сайт.

В этой задаче тебе нужно будет в каждый день найти репетитора, уроки которого получили самую низкую оценку качества, и вывести эту оценку.

В архиве (https://yadi.sk/d/ESH_lU5sZRdKhA) содержится четыре файла с выгрузкой строк из базы.

lessons.txt содержит следующие поля:
    id – уникальный идентификатор урока.
    event_id – идентификатор, связывающий уроки с файлом participants. У нескольких уроков может быть один и тот же event_id. Подробнее про это будет позже
    subject – предмет урока, просто строка
    scheduled_time – время начала урока, в формате ГГГГ-ММ-ДД чч:мм:сс (иногда есть еще миллисекунды). Время указано в UTC.

quality.txt содержит следующие поля:
    lesson_id – идентификатор урока, указывает на запись в таблице lessons.txt. У нескольких строчек из этого файла может быть один и тот же lesson_id, потому что оценок за урок может быть несколько (например, оба участника, ученик и репетитор могут оценить качество урока)
    tech_quality – собственно оценка качества урока. Это число от 1 до 5. Иногда его может не быть, если пользователь не выставил оценку.

users.txt хранит информацию о пользователях и содержит всего два поля:
    id – уникальный идентификатор пользователя
    role – указывает является ли пользователь учеником (pupil) или учителем (tutor)

Файл participants.txt позволяет связать урок с его участниками. Он содержит следующие поля:
    user_id – идентификатор пользователя (указывает на запись в файле users.txt)
    event_id – идентификатор, связывающий урок с участником. То есть, чтобы понять, какие пользователи были на уроке Х, нужно найти в файле participants.txt строки, у которых event_id совпадает с event_id урока Х.

Для решения задачки нужно сделать следующее:
1. Найти все уроки по физике (subject=phys).


In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
with open('lessons.txt', encoding='utf8') as f: 
    lessons = pd.read_csv('lessons.txt',sep='|', header=None, names=["id", "event_id", "subject", "scheduled_time"]) 

In [4]:
lessons = lessons.drop([0, 1, 380])

In [5]:
lessons = lessons.astype({'event_id': 'int64'})

In [6]:
lessons['scheduled_time'] = pd.to_datetime(lessons['scheduled_time'])

In [7]:
lessons.dtypes

id                        object
event_id                   int64
subject                   object
scheduled_time    datetime64[ns]
dtype: object

In [8]:
lessons['id'] = lessons['id'].replace(r'\s*','',regex=True)

In [9]:
lessons['subject'] = lessons['subject'].replace(r'\s*','',regex=True)

In [10]:
with open('quality.txt', encoding='utf8') as f: 
    quality = pd.read_csv('quality.txt',sep='|', header=None, names=["lesson_id", "tech_quality"]) 

In [11]:
quality = quality.drop([0,1,367])

In [12]:
quality['tech_quality'] = pd.to_numeric(quality['tech_quality'], errors='coerce', downcast='unsigned')

In [13]:
quality['lesson_id'] = quality['lesson_id'].replace(r'\s*','',regex=True)

In [14]:
with open('users.txt', encoding='utf8') as f: 
    users = pd.read_csv('users.txt',sep='|', header=None, names=["id", "role"]) 
users = users.drop([0,1,745])

In [15]:
users['id'] = users['id'].replace(r'\s*','',regex=True)
users['role'] = users['role'].replace(r'\s*','',regex=True)

In [16]:
with open('participants.txt', encoding='utf8') as f: 
    participants = pd.read_csv('participants.txt',sep='|', header=None, names=["event_id", "user_id"]) 
participants = participants.drop([0,1,745])

In [17]:
participants = participants.astype({'event_id': 'int64'});

In [18]:
participants['user_id'] = participants['user_id'].replace(r'\s*','',regex=True)

### объединяем

In [19]:
part_use = pd.merge(users, participants,  left_on='id', right_on='user_id')

In [20]:
part_use = part_use.drop(columns='id')

In [21]:
lessons_qua = pd.merge(lessons, quality, left_on='id', right_on=['lesson_id'], how='outer')
lessons_qua = lessons_qua.drop(columns='lesson_id')
lessons_qua = lessons_qua.set_index(['event_id', 'id'])

In [22]:
last = pd.merge(participants, lessons, on='event_id', how='right')
last = last.drop_duplicates()

In [23]:
last = last.set_index(['event_id', 'user_id'])

In [24]:
part_use = part_use.set_index(['event_id', 'user_id'])

In [25]:
last_part = pd.merge(last, part_use, on=['event_id', 'user_id'], how='left')
last_part = last_part.reset_index()
last_part = last_part.drop_duplicates()

In [26]:
last_part = last_part.set_index(['event_id', 'id'])

In [27]:
all_data = pd.merge(last_part, lessons_qua['tech_quality'], on=['event_id', 'id'], how='left')
all_data = all_data.reset_index()
all_data = all_data.drop_duplicates()
all_data

,event_id,id,user_id,subject,scheduled_time,role,tech_quality
0,38114,1e7bb408-cfef-4a9f-8328-351c9483a64c,e28351f5-4ccb-4549-8647-d43f2b15e7b8,phys,2020-01-19 12:00:00,pupil,NaN
1,38114,1e7bb408-cfef-4a9f-8328-351c9483a64c,4df2832a-1d63-4453-9659-43993fc35996,phys,2020-01-19 12:00:00,tutor,NaN
2,38114,2e3b1173-7edb-4737-9500-a23d7b852518,e28351f5-4ccb-4549-8647-d43f2b15e7b8,phys,2020-01-12 12:00:00,pupil,5.0
3,38114,2e3b1173-7edb-4737-9500-a23d7b852518,4df2832a-1d63-4453-9659-43993fc35996,phys,2020-01-12 12:00:00,tutor,5.0
4,51568,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,bb1c0bc8-1212-452b-97a0-439d4a2169e2,it,2020-01-19 13:00:00,pupil,5.0
...,...,...,...,...,...,...,...
974,51001,056c3d91-dce7-4071-91d3-1de816ee63a3,6a578a6e-39a4-4c6b-8b0a-f00f6b2dcd79,hist,2020-01-17 12:45:00,tutor,4.0
975,49880,aa7702bc-5271-469c-a1c2-b7e814fd7e20,8476b612-223f-4c86-9b0b-14bc04759233,bio,2020-01-16 15:00:00,tutor,5.0
976,49880,aa7702bc-5271-469c-a1c2-b7e814fd7e20,582e6abb-dd9a-42fa-b441-455eac28327e,bio,2020-01-16 15:00:00,pupil,5.0
977,55449,8643fa00-217a-42ae-a27d-e6f47c0501f1,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb,bio,2020-01-19 12:15:00,tutor,5.0


### так как в таблице оценок не прописан id того кто ставит оценку, 
таблицу all_data и lessons_qua использую совместно, для корректировки

In [28]:
all_data_phys = all_data.loc[all_data['subject'] == 'phys']
all_data_phys

,event_id,id,user_id,subject,scheduled_time,role,tech_quality
0,38114,1e7bb408-cfef-4a9f-8328-351c9483a64c,e28351f5-4ccb-4549-8647-d43f2b15e7b8,phys,2020-01-19 12:00:00,pupil,NaN
1,38114,1e7bb408-cfef-4a9f-8328-351c9483a64c,4df2832a-1d63-4453-9659-43993fc35996,phys,2020-01-19 12:00:00,tutor,NaN
2,38114,2e3b1173-7edb-4737-9500-a23d7b852518,e28351f5-4ccb-4549-8647-d43f2b15e7b8,phys,2020-01-12 12:00:00,pupil,5.0
3,38114,2e3b1173-7edb-4737-9500-a23d7b852518,4df2832a-1d63-4453-9659-43993fc35996,phys,2020-01-12 12:00:00,tutor,5.0
16,51341,00fc6685-f53a-49bb-b960-5e0042fd3852,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2,phys,2020-01-17 12:00:00,pupil,5.0
...,...,...,...,...,...,...,...
962,55490,e90f529b-1061-4efd-9bba-f73ecee2ef27,43efce48-94b2-4412-857f-223d45969008,phys,2020-01-19 08:00:00,tutor,4.0
963,55490,e90f529b-1061-4efd-9bba-f73ecee2ef27,b323a494-40c0-4522-8d63-4d4d591a1def,phys,2020-01-19 08:00:00,pupil,5.0
964,55490,e90f529b-1061-4efd-9bba-f73ecee2ef27,b323a494-40c0-4522-8d63-4d4d591a1def,phys,2020-01-19 08:00:00,pupil,4.0
965,55881,8a1bac54-c5c6-47c9-8fdb-36d01384eaae,c6718d0e-976c-4d6c-b0e0-32c770776567,phys,2020-01-19 15:00:00,tutor,NaN


2. В каждый день (начало и конец дня считается по московскому времени, то есть UTC+3:00) для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из п. 1). То есть, если учитель проводил в этот день три урока по физике, один из них он оценил на 3, а ученик оценил его на 4, второму уроку оценку поставил только ученик, и эта оценка 5, а третий урок вообще никто не оценил, то средняя арифметическая оценка учителя за уроки = (3 + 4 + 5) / 3 = 4.


In [29]:
all_data_phys['ave'] = 0

In [30]:
all_data_phys['scheduled_time'] = all_data_phys.scheduled_time.dt.tz_localize(tz='Europe/Moscow')

In [31]:
all_data_phys['dateonly'] = all_data_phys['scheduled_time'].apply(lambda x: x.date()) 

In [32]:
all_data_phys = all_data_phys.set_index(['dateonly', 'id'])

In [33]:
all_data__phys_teachers = all_data_phys.loc[all_data_phys['role'] == 'tutor']
all_data__phys_teachers = all_data__phys_teachers.reset_index()
all_data__phys_teachers = all_data__phys_teachers.set_index(['dateonly', 'user_id'])

In [34]:
indexes = all_data__phys_teachers.index.unique()

In [35]:
indexes_two = []
num = 0
for i in indexes:    
    indexes_two.append([])
    bp = all_data__phys_teachers.loc[i, ['id']]
    bp = bp.reset_index()
    bp = bp.set_index(['dateonly', 'user_id', 'id'])
    bp['o'] = 0
    for j in bp.index:
        if j not in indexes_two[num]:
            indexes_two[num].append(j)
    num+=1

In [36]:
hm = []
num = 0
for k in indexes_two:
    hm.append([])
    for v in k:
        l = (v[0], v[2])
        hm[num].append(l)
    num+=1

In [37]:
lessons_qua['dateonly'] = lessons_qua['scheduled_time'].apply(lambda x: x.date())
lessons_qua = lessons_qua.reset_index()

In [38]:
lessons_qua = lessons_qua.set_index(['dateonly', 'id'])

In [39]:
for i in hm:    
    k = lessons_qua.loc[i]
    m = k['tech_quality'].dropna().mean()
    all_data_phys.loc[i, 'ave'] = m

In [40]:
all_data__phys_teachers = all_data_phys.loc[all_data_phys['role'] == 'tutor']
all_data__phys_teachers = all_data__phys_teachers.drop(columns=['tech_quality'])
all_data__phys_teachers = all_data__phys_teachers.reset_index().set_index('dateonly').drop_duplicates()
all_data__phys_teachers

,id,event_id,user_id,subject,scheduled_time,role,ave
dateonly,,,,,,,
2020-01-19,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,4df2832a-1d63-4453-9659-43993fc35996,phys,2020-01-19 12:00:00+03:00,tutor,5.000
2020-01-12,2e3b1173-7edb-4737-9500-a23d7b852518,38114,4df2832a-1d63-4453-9659-43993fc35996,phys,2020-01-12 12:00:00+03:00,tutor,5.000
2020-01-17,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,30a19496-bdaf-461c-abbc-2709ae520201,phys,2020-01-17 12:00:00+03:00,tutor,5.000
2020-01-19,4cadf623-82e6-422f-a342-acf978302fb2,55048,43efce48-94b2-4412-857f-223d45969008,phys,2020-01-19 14:00:00+03:00,tutor,4.875
2020-01-19,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,43efce48-94b2-4412-857f-223d45969008,phys,2020-01-19 15:00:00+03:00,tutor,4.875
...,...,...,...,...,...,...,...
2020-01-19,e0d57734-07b0-4158-b442-716955f6175c,53439,8fe03f08-8581-430c-a590-9888ab36deb3,phys,2020-01-19 07:00:00+03:00,tutor,5.000
2020-01-19,2f3c2e0a-e188-420e-ae83-8673ea692d66,47780,8fe03f08-8581-430c-a590-9888ab36deb3,phys,2020-01-19 06:00:00+03:00,tutor,5.000
2020-01-20,459ed706-9310-477f-9f51-7a290b28b6a8,53917,30a19496-bdaf-461c-abbc-2709ae520201,phys,2020-01-20 08:00:00+03:00,tutor,4.500


3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).


In [41]:
all_data__phys_teachers = all_data__phys_teachers.reset_index().set_index(['dateonly', 'user_id'])
all_data__phys_teachers_ave = all_data__phys_teachers.ave

In [42]:
all_data__phys_teachers_ave = all_data__phys_teachers_ave.reset_index()
all_data__phys_teachers_ave.drop_duplicates(inplace=True)

In [43]:
all_data__phys_teachers_ave.sort_values(['dateonly', 'ave'], ascending = True , inplace=True)

In [44]:
min_av = all_data__phys_teachers_ave.groupby(['dateonly'])['ave'].min()
min_av = min_av.reset_index().set_index(['dateonly','ave'])
min_av_in = min_av.index

In [45]:
all_data__phys_teachers_ave = all_data__phys_teachers_ave.set_index(['dateonly', 'ave'])

In [46]:
index_col = ['dateonly', 'ave', 'user_id']
result = pd.DataFrame(columns=index_col)
for i in min_av_in:
    j = all_data__phys_teachers_ave.loc[i]
    result = pd.merge(result, j.reset_index(), how='outer')

4. Вывести его в формате "<день> <id учителя> <средняя арифметическая оценка>". Оценку можно округлить с точностью до двух знаков после запятой. То есть как-то так:
2020-01-11 73c9af08-8581-430c-a590-9888ab36deb3 3.67 2020-01-12 909c2c8e-c054-4e9f-a51a-50bf5660f364 3.25
...
4. Учитывать нужно только тех учителей, за уроки по физике которых в этот день стоит хотя бы одна оценка. Если у нескольких учителей одна и та же самая оценка, можно вывести любого.

In [47]:
result = result.set_index(['dateonly','user_id'])

In [48]:
result =  result.round({'ave':2}) 
result

ave
dateonly   user_id                                   
2020-01-11 8fe03f08-8581-430c-a590-9888ab36deb3  4.43
2020-01-12 696c838e-c054-4e9f-a51a-50bf5660f364  4.89
2020-01-13 696c838e-c054-4e9f-a51a-50bf5660f364  5.00
           30a19496-bdaf-461c-abbc-2709ae520201  5.00
           b37ccae8-fc31-4ad8-8f55-ca855b23fbf6  5.00
           2fa2ab62-f1b0-4036-872f-bcfd9a8686ff  5.00
           be676776-8366-4c71-8a35-d58014806eb5  5.00
2020-01-14 c6718d0e-976c-4d6c-b0e0-32c770776567  4.00
2020-01-15 b37ccae8-fc31-4ad8-8f55-ca855b23fbf6  5.00
           603b8641-c6f6-4d89-ac89-88e50d45aa0d  5.00
           696c838e-c054-4e9f-a51a-50bf5660f364  5.00
           9b26952c-3470-4032-a9cd-f1e9d4f90ece  5.00
2020-01-16 2fa2ab62-f1b0-4036-872f-bcfd9a8686ff  4.00
2020-01-17 696c838e-c054-4e9f-a51a-50bf5660f364  4.50
2020-01-18 43efce48-94b2-4412-857f-223d45969008  4.25
2020-01-19 be676776-8366-4c71-8a35-d58014806eb5  4.50
2020-01-20 30a19496-bdaf-461c-abbc-2709ae520201  4.50
           43efce48-94b2-4412-857f-223d45969008  4.50

In [49]:
result.reset_index()

,dateonly,user_id,ave
0,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.43
1,2020-01-12,696c838e-c054-4e9f-a51a-50bf5660f364,4.89
2,2020-01-13,696c838e-c054-4e9f-a51a-50bf5660f364,5.00
3,2020-01-13,30a19496-bdaf-461c-abbc-2709ae520201,5.00
4,2020-01-13,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.00
5,2020-01-13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
6,2020-01-13,be676776-8366-4c71-8a35-d58014806eb5,5.00
7,2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.00
8,2020-01-15,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.00
9,2020-01-15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.00
